# Imports 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('final_data.csv')

In [3]:
df

,video_id,title,split_comments,num_tokens,sentiment,labels,scores
0,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,how to remember the system then?,7,neutral,"['aspirations', 'frustrations', 'fears']","[0.45186224579811096, 0.34655919671058655, 0.2..."
1,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,"15/3/24, x2.",9,neutral,"['aspirations', 'fears', 'frustrations']","[0.36963728070259094, 0.3612099289894104, 0.26..."
2,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,thanks brother it's helping video :red_heart::...,16,positive,"['aspirations', 'frustrations', 'fears']","[0.6822587251663208, 0.17578278481960297, 0.14..."
3,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,you should try breathing and just being in the...,28,neutral,"['frustrations', 'aspirations', 'fears']","[0.4940927028656006, 0.31500494480133057, 0.19..."
4,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,took me out when he said “my brain is a dumass...,52,negative,"['frustrations', 'fears', 'aspirations']","[0.8241838812828064, 0.0920601338148117, 0.083..."
...,...,...,...,...,...,...,...
14921,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,please also upload hindi video,6,neutral,"['fears', 'aspirations', 'frustrations']","[0.3803737759590149, 0.32163533568382263, 0.29..."
14922,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,im 3rd,3,neutral,"['aspirations', 'frustrations', 'fears']","[0.6356985569000244, 0.18799357116222382, 0.17..."
14923,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,what if he pinned my cmt?:grinning_face_with_s...,18,neutral,"['fears', 'frustrations', 'aspirations']","[0.5885409116744995, 0.3420863449573517, 0.069..."
14924,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,first to comment,3,neutral,"['aspirations', 'fears', 'frustrations']","[0.4376871585845947, 0.28964871168136597, 0.27..."


# Identifying biggest frustrations 

In [10]:
(df[['labels','scores']].iloc[0:5])

,labels,scores
0,"['aspirations', 'frustrations', 'fears']","[0.45186224579811096, 0.34655919671058655, 0.2..."
1,"['aspirations', 'fears', 'frustrations']","[0.36963728070259094, 0.3612099289894104, 0.26..."
2,"['aspirations', 'frustrations', 'fears']","[0.6822587251663208, 0.17578278481960297, 0.14..."
3,"['frustrations', 'aspirations', 'fears']","[0.4940927028656006, 0.31500494480133057, 0.19..."
4,"['frustrations', 'fears', 'aspirations']","[0.8241838812828064, 0.0920601338148117, 0.083..."


In [14]:
# get fears etc standardised and scored also 
desired_order = ['fears', 'frustrations', 'aspirations']


In [15]:
# Function to reorder the scores based on the desired order of labels
def reorder_scores(row):
    label_score_dict = dict(zip(row['labels'], row['scores']))
    return [label_score_dict[label] for label in desired_order]


In [39]:
df['reordered_labels'] = [desired_order] * len(df)
df['reordered_scores'] = df.apply(reorder_scores, axis=1)

KeyError: 'fears'

In [41]:
def reorder_scores(labels, scores, desired_order):
    label_score_dict = dict(zip(labels, scores))
    reordered_scores = [label_score_dict.get(label, 0) for label in desired_order]
    return reordered_scores

In [42]:
desired_label_order = ['fears', 'frustrations', 'aspirations']

In [43]:
df['reordered_scores'] = df.apply(lambda row: reorder_scores(row['labels'], row['scores'], desired_label_order), axis=1)

In [45]:
df.head()

,video_id,title,split_comments,num_tokens,sentiment,labels,scores,reordered_labels,reordered_scores
0,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,how to remember the system then?,7,neutral,"['aspirations', 'frustrations', 'fears']","[0.45186224579811096, 0.34655919671058655, 0.2...","[fears, frustrations, aspirations]","[0, 0, 0]"
1,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,"15/3/24, x2.",9,neutral,"['aspirations', 'fears', 'frustrations']","[0.36963728070259094, 0.3612099289894104, 0.26...","[fears, frustrations, aspirations]","[0, 0, 0]"
2,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,thanks brother it's helping video :red_heart::...,16,positive,"['aspirations', 'frustrations', 'fears']","[0.6822587251663208, 0.17578278481960297, 0.14...","[fears, frustrations, aspirations]","[0, 0, 0]"
3,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,you should try breathing and just being in the...,28,neutral,"['frustrations', 'aspirations', 'fears']","[0.4940927028656006, 0.31500494480133057, 0.19...","[fears, frustrations, aspirations]","[0, 0, 0]"
4,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,took me out when he said “my brain is a dumass...,52,negative,"['frustrations', 'fears', 'aspirations']","[0.8241838812828064, 0.0920601338148117, 0.083...","[fears, frustrations, aspirations]","[0, 0, 0]"


In [46]:
data = pd.read_csv('reorderd_data.csv')

In [47]:
data

,video_id,title,split_comments,num_tokens,sentiment,labels,scores,reordered_scores
0,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,how to remember the system then?,7,neutral,"['aspirations', 'frustrations', 'fears']","[0.45186224579811096, 0.34655919671058655, 0.2...","[0.20157860219478607, 0.34655919671058655, 0.4..."
1,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,"15/3/24, x2.",9,neutral,"['aspirations', 'fears', 'frustrations']","[0.36963728070259094, 0.3612099289894104, 0.26...","[0.3612099289894104, 0.26915282011032104, 0.36..."
2,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,thanks brother it's helping video :red_heart::...,16,positive,"['aspirations', 'frustrations', 'fears']","[0.6822587251663208, 0.17578278481960297, 0.14...","[0.14195847511291504, 0.17578278481960297, 0.6..."
3,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,you should try breathing and just being in the...,28,neutral,"['frustrations', 'aspirations', 'fears']","[0.4940927028656006, 0.31500494480133057, 0.19...","[0.19090235233306885, 0.4940927028656006, 0.31..."
4,T6hmdrsLQj8,How to Build Your Ultimate Productivity System,took me out when he said “my brain is a dumass...,52,negative,"['frustrations', 'fears', 'aspirations']","[0.8241838812828064, 0.0920601338148117, 0.083...","[0.0920601338148117, 0.8241838812828064, 0.083..."
...,...,...,...,...,...,...,...,...
14921,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,please also upload hindi video,6,neutral,"['fears', 'aspirations', 'frustrations']","[0.3803737759590149, 0.32163533568382263, 0.29...","[0.3803737759590149, 0.29799097776412964, 0.32..."
14922,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,im 3rd,3,neutral,"['aspirations', 'frustrations', 'fears']","[0.6356985569000244, 0.18799357116222382, 0.17...","[0.17630787193775177, 0.18799357116222382, 0.6..."
14923,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,what if he pinned my cmt?:grinning_face_with_s...,18,neutral,"['fears', 'frustrations', 'aspirations']","[0.5885409116744995, 0.3420863449573517, 0.069...","[0.5885409116744995, 0.3420863449573517, 0.069..."
14924,3bamBYQS8io,5 Small Changes to Supercharge Your Focus,first to comment,3,neutral,"['aspirations', 'fears', 'frustrations']","[0.4376871585845947, 0.28964871168136597, 0.27...","[0.28964871168136597, 0.2726641297340393, 0.43..."
